In [1]:
# !pip3 install html5lib

In [47]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlencode
import urllib.parse

# Scrape index


In [102]:
# Define some functions to help scrape the index

def get_osha_url(query, start_date_str, end_date_str):
    
    results_per_page = 1000
    start_date = pd.to_datetime(start_date_str)
    end_date = pd.to_datetime(end_date_str)

    # make params dictionary
    base_url = f"https://www.osha.gov/ords/imis/establishment.search"
    params = {
        'establishment': '+'.join(query.split()),
        'state': "all",
        'officetype': "all",
        'office': "all",
        'sitezip': 100000,
        'startmonth': start_date.month,
        'startday': start_date.day,
        'startyear': start_date.year,
        'endmonth': end_date.month,
        'endday': end_date.day,
        'endyear': end_date.year,
        'p_case': "all",
        'p_sort': 12,
        'p_desc': "DESC",
        'p_direction': "Prev",
        'p_show': results_per_page,
        'p_violations_exist': "yes"
    }

    return base_url + '?' + urlencode(params)

def get_index_table(osha_url):
    return pd.read_html(osha_url)[2]
    

In [103]:
queries = [
    ('Dollar General', '2012-11-18', '2022-11-18'),
    ('Dollar Tree', '2012-11-18', '2022-11-18')
]

In [119]:
activities_df = []
for query in queries:
    
    # break the query into 3 variables, and pass those to get_osha_url
    establishment, start, end = query   
    osha_url = get_osha_url(establishment, start, end)
    print(osha_url)
    
    # get the index table from the osha url, drop a junk column    
    index_df_for_this_query = get_index_table(osha_url)
    index_df_for_this_query = index_df_for_this_query.drop(columns='Unnamed: 0')
    
    # tack on the query parameters to the table
    index_df_for_this_query['query'] = establishment
    index_df_for_this_query['query_start'] = start
    index_df_for_this_query['query_end'] = end
    
    # print to verify that pagination isn't a problem 
    # IMPORTANT: 👀 look at these values to make sure we don't need to paginate     
    print(f"found {len(index_df_for_this_query)} results for {query}")

    # append to dataframe
    activities_df.append(index_df_for_this_query)
    
# combine the queries
activities_df = pd.concat(activities_df)
activities_df.to_csv('activities_df.csv', index=False)
activities_df

https://www.osha.gov/ords/imis/establishment.search?establishment=Dollar%2BGeneral&state=all&officetype=all&office=all&sitezip=100000&startmonth=11&startday=18&startyear=2012&endmonth=11&endday=18&endyear=2022&p_case=all&p_sort=12&p_desc=DESC&p_direction=Prev&p_show=1000&p_violations_exist=yes
found 136 results for ('Dollar General', '2012-11-18', '2022-11-18')
https://www.osha.gov/ords/imis/establishment.search?establishment=Dollar%2BTree&state=all&officetype=all&office=all&sitezip=100000&startmonth=11&startday=18&startyear=2012&endmonth=11&endday=18&endyear=2022&p_case=all&p_sort=12&p_desc=DESC&p_direction=Prev&p_show=1000&p_violations_exist=yes
found 377 results for ('Dollar Tree', '2012-11-18', '2022-11-18')


,#,Activity,Opened,RID,St,Type,Sc,SIC,NAICS,Vio,Establishment Name,query,query_start,query_end
0,1,1.617010e+06,08/25/2022,552652,MI,Complaint,Partial,NaN,452319,1,Dollar General #17771,Dollar General,2012-11-18,2022-11-18
1,2,1.609906e+06,07/25/2022,454716,TN,Complaint,Partial,NaN,452319,2,Dollar General Corporation,Dollar General,2012-11-18,2022-11-18
2,3,1.607451e+06,07/13/2022,418100,GA,Complaint,Partial,NaN,445110,1,Dollar General Atlanta Fresh Distribution Center,Dollar General,2012-11-18,2022-11-18
3,4,1.599901e+06,06/03/2022,418600,FL,Complaint,Partial,NaN,452319,3,"Dollar General Corporation/ Dolgencorp, Llc",Dollar General,2012-11-18,2022-11-18
4,5,1.598544e+06,05/24/2022,418200,GA,Complaint,Partial,NaN,452319,7,"Dollar General Corporation/ Dolgencorp, Llc",Dollar General,2012-11-18,2022-11-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372,373,3.166403e+08,06/18/2013,1054113,OR,Planned,Complete,5331.0,452990,2,Dollar Tree Stores Inc,Dollar Tree,2012-11-18,2022-11-18
373,374,3.162124e+08,06/13/2013,950633,CA,Complaint,Partial,5331.0,452990,3,The Dollar Tree Store #3621,Dollar Tree,2012-11-18,2022-11-18
374,375,3.168030e+08,05/28/2013,1055340,WA,Referral,Partial,5399.0,452990,1,Dollar Tree Stores # 2264,Dollar Tree,2012-11-18,2022-11-18
375,376,3.166980e+08,04/16/2013,950624,CA,Complaint,Partial,5331.0,452990,2,"Dollar Tree Stores, Inc Dba Dollar Tree #4403",Dollar Tree,2012-11-18,2022-11-18


# Scrape individual activity pages

In [120]:
# Figure out how to scrape individual activity
def get_related_activity_table(inspection_url):
    tables = pd.read_html(inspection_url)
    return tables[0]

def get_violation_summary_table(inspection_url):
    tables = pd.read_html(inspection_url)
    return tables[1]

def get_violation_items_table(inspection_url):
    tables = pd.read_html(inspection_url)
    return tables[2]

def get_details(inspection_url):
    
    # get HTML from inspection page
    response = requests.get(inspection_url)
    html_doc = response.text
    soup = BeautifulSoup(html_doc, 'html.parser')

    # get main container (ignore footer/header/etc...)
    html_main_container = soup.find(id="maincontain")

    # get case status
    html_wells = html_main_container.find_all("div", class_="well")
    assert len(html_wells) == 2
    assert html_wells[0] == html_wells[1]
    case_status = html_wells[0].text
    
    # get details
    details = {}
    html_spans = html_main_container.find_all("div", class_="span4")
    for span in html_spans:
        html_p_tags = span.find_all('p')
        if len(html_p_tags) == 0:
            key = span.text.split(':')[0].strip()
            value = span.text.split(':')[1].strip()
            details[key] = value
        else:
            for p_tag in html_p_tags:
                key = p_tag.text.split(':')[0].strip()
                value = p_tag.text.split(':')[1].strip()
                details[key] = value
    
    # add case status to details
    details['Case Status'] = case_status
    
    return details
    

In [116]:
violation_summary_tables = []
violation_items_tables = []
related_activity_tables = []
details_dictionaries = []

for index, row in activities_df.head(5).iterrows():
    activity_code = row['Activity']
    url = f"https://www.osha.gov/ords/imis/establishment.inspection_detail?id={activity_code}"
    print(f"{index} of {len(index_df)} - scraping {url}")

    violation_summary = get_violation_summary_table(url)
    violation_summary['activity_code'] = activity_code
    violation_summary_tables.append(violation_summary)
    
    violation_items = get_violation_items_table(url)
    violation_items['activity_code'] = activity_code
    violation_items_tables.append(violation_items)

    related_activity = get_related_activity_table(url)
    related_activity['activity_code'] = activity_code
    related_activity_tables.append(related_activity)

    details = get_details(url)
    details['activity_code'] = activity_code
    details_dictionaries.append(details)


0 of 513 - scraping https://www.osha.gov/ords/imis/establishment.inspection_detail?id=1617010.015
1 of 513 - scraping https://www.osha.gov/ords/imis/establishment.inspection_detail?id=1609906.015
2 of 513 - scraping https://www.osha.gov/ords/imis/establishment.inspection_detail?id=1607451.015
3 of 513 - scraping https://www.osha.gov/ords/imis/establishment.inspection_detail?id=1599901.015
4 of 513 - scraping https://www.osha.gov/ords/imis/establishment.inspection_detail?id=1598544.015


In [117]:
violation_summary_df = pd.concat(violation_summary_tables)
violation_items_df = pd.concat(violation_items_tables)
related_activity_df = pd.concat(related_activity_tables)
details_df = pd.DataFrame(details_dictionaries)

In [118]:
violation_summary_df.to_csv('violation_summary_df.csv', index=False)
violation_items_df.to_csv('violation_items_df.csv', index=False)
related_activity_df.to_csv('related_activity_df.csv', index=False)
details_df.to_csv('details_df.csv', index=False)